# 2 - Redes recurrentes orientadas al procesamiento de lenguaje natural

**Sumario**

1. Introducción
2. El framewok de aprendizaje por refuerzo
3. Deep Reinforcement Learning
4. Construyendo un agente mediante una red DQNN

## 2.1 - Introducción

Cuando reflexionamos sobre la palabra **aprendizaje**, es posible que lo primero que nos venga a la mente sea **la forma en la que los seres humanos aprendemos mediante la interacción con nuestro entorno**.

Así, uno de los primeros ejemplos que seguramente visualicemos sea el proceso de aprendizaje que experimentan los niños y las niñas cuando empiezan a andar, donde interactúan con el entorno mediante un modelo de causa y efecto que voluciona con experimentación. De este modo, los seres humanos vamos adaptando poco a poco nuestro sistema motriz cada vez que se producen caidas (**"penalizan" el modelo**) o que conseguimos alcanzar una determinada localización en nuestro entorno (**"refuerzan"** el modelo).

<table>
    <tr>
        <td><img src="images_2/humans.jpg" width="500" data-align="center"></td>
    </tr>
</table>

Dicho sistema de **penalización** y **refuerzo** se utilizó como inspiración para definir **el aprendizaje por refuerzo** (Sutton, 1998). El cual define un modelo de comportamiento que simula el proceso de interacción entre un agente y un entorno mediante la utilización de estímulos (refuerzos) que le indicasen cuales son las decisiones mas prometedoras para alcanzar un determinado objetivo.

Comenzó con acermientos basados en **programación dinámica**, y luego fue evolucionando hacia los **algoritmos basados en tablas**, como *Q-Learning* o *SARSA*, los cuales construyen una tabla (normalmente llamada **tabla Q**) donde para cada estado, se incluía un valor de calidad para cada una de las posibles acciones que se podrian realizar en un determinado estado.

Este tipo de acercamientos implicaban un gran **problema computacional** ya que hacia que muchos problemas resultasen intratables por la **combinación de estados y acciones que definía el tamaño de la tabla**. Este inconveniente se solventó con el surgimiento del aprendizaje profundo (*Deep Learning*), permitiendo que el aprendizaje por refuerzo pudiera resolver problemas antes inabordables, es decir, problemas donde el tamaño de la tabla Q seria demasiado grande.

En el **aprendizaje por refuerzo profundo** (***Deep Reinforcement Learning***) utilizamos una red neuronal en vez de una tabla Q para dar un valor "aproximado" de la calidad de una acción para un determinado estado. Este tipo de acercamiento fue un gran avance en el área, permitiendo desarrollar modelos para tareas impensables hasta el momento, como videojuegos (e.g., [**Atari**](https://openai.com/research/gym-retro) o [**Starcraft II**](https://www.deepmind.com/blog/alphastar-grandmaster-level-in-starcraft-ii-using-multi-agent-reinforcement-learning)), [**el juego de Go**](https://www.youtube.com/watch?v=WXuK6gekU1Y) o [**el plegamiento de proteinas**](https://www.deepmind.com/research/highlighted-research/alphafold).

<table>
    <tr>
        <td><img src="images_2/alphago.jpg" width="500" data-align="center"></td>
    </tr>
</table>

A lo largo de esta unidad, describiremos los conceptos básicos asociados al aprendizaje por refuerzo y estudiaremos cómo este ha evolucionado desde los algoritmos tradicionales hasta las redes de neuronas por refuerzo, que utilizaremos, finalmente, para construir un agente que aprenda a jugar a un videojuego.

## 2.2 - El framework de aprendizaje por refuerzo

La idea detrás del aprendizaje por refuerzo es que un **agente** (una IA) aprenderá del **entorno** interactuando con él (a través de prueba y error) y recibiendo **recompensas** (negativas o positivas) como retroalimentación por realizar acciones.

Por ejemplo, imagina poner a tu hermano pequeño frente a un videojuego que nunca jugó, darle un controlador y dejarlo solo. Tu hermano podria interactuar con el "entorno" (el videojuego) pulsando los botones del mando. Si obtiene una moneda, recibirá un recompensa positivo (e.g., +1). En cambio, si toca un enemigo, recibirá una recompensa negativa (e.g., -1).

Al interactuar con su entorno a través de prueba y error, tu hermano acabaría entendiendo que necesita recolectar monedas y evitar enemigos. Sin ninguna supervisión el niño mejoraría  cada vez más en el juego.

<table>
    <tr>
        <td><img src="images_2/RL_process_game.jpg" width="500" data-align="center"></td>
    </tr>
</table>

* **Agente**. La entidad que interactúa con el entorno obteniendo información de éste y modificándolo  mediante acciones.
* **Entorno**. La representación virtual del mundo con el que interactúa el agente.
* **Estado**. La representación completa del entorno en un instante específico de tiempo, así como la representación del agente en dicho instante.
* **Acción**. Aquella que ejecuta el agente ene el entorno con el fin de producir una variación en este.
* **Recompensa o refuerzo**. Valor numérico obtenido tras la ejecución de una acción en el entorno. Se utiliza como medida para evaluar si la ejecución de la acción ha resultado positiva o negativa para el agente

**Este proceso es un bucle, donde el objetivo del agente es maximizar la recompensa acumulada, que representa el retorno esperado**.

Hay varios componentes/asunciones relevantes en el framework del aprendizaje por refuerzo (los veremos ahora uno por uno):
* La hipótesis de recompensa.
* La propiedad de Markov.
* El grado de observabilidad del entorno (total o parcial).
* El espacio de acción (finito o infinito).
* Las recompensa y el factor de descuento.
* Enfoque de aprendizaje (basado en política o en valor)

### 2.2.1 - La hipótesis de recompensa

El aprendizaje por refuerzo se basa en la **hipótesis de recompensa**, la cual indica que **cualquier meta puede describirse como la maximización del rendimiento esperado** (recompensa acumulada esperada).

Es por eso que en el aprendizaje por refuerzo, para tener **el mejor comportamiento**, nuestro objetivo es aprender a **tomar acciones que maximicen la recompensa acumulada esperada**.

### 2.2.2 - La propiedad de Markov

El proceso de RL también se denomina **Proceso de decisión de Markov** (*Markov Decission Process*, *MDP* por sus siglas en inglés).

La propiedad de Markov implica que **nuestro agente solo necesita el estado actual para decidir qué acción tomar** y no el historial de todos los estados y acciones que tomó antes.

### 2.2.3 - El grado de observabilidad del entorno

<table>
    <tr>
        <td><img src="images_2/obs_space.jpg" width="600" data-align="center"></td>
    </tr>
</table>

-----

**Nota**: Ciertos autores hacen la distinción entre "estado" y "observación" para nombrar a $S_{t}$ en el bucle de aprendizaje por refuerzo.

-----

### 2.2.4 - El espacio de acción

<table>
    <tr>
        <td><img src="images_2/action_space.jpg" width="600" data-align="center"></td>
    </tr>
</table>

### 2.2.5 - La recompensa y el factor de descuento

La recompensa es un elemento fundamental en el proceso de aprendizaje automático porque **es el único feedback que recibe nuestro agente**. Gracias a ella, sabe si esta acción ha sido **positiva o negativa**.

La recompensa acumulada para un instante de tiempo $t$ se puede escribir como:

<table>
    <tr>
        <td><img src="images_2/rewards_1.png" width="400" data-align="center"></td>
    </tr>
</table>

Esto es equivalente a:

$$
R(\tau) = \sum_{k=0}^{\infty} r_{t+k+1}
$$

Sin embargo, en realidad, no podemos simplemente agregar las recompensas así. Las recompensas que llegan antes (al comienzo del juego) **tienen más probabilidades de suceder**, ya que son más predecibles que las recompensas futuras a largo plazo.

Digamos que tu agente es este pequeño ratón que puede mover una ficha en cada paso de tiempo, y tu oponente es el gato (que también puede moverse). **El objetivo del ratón es comer la máxima cantidad de queso antes de ser comido por el gato.**

En consecuencia, la recompensa cerca del gato, aunque sea más grande (más queso), **estará más rebajada ya que no estamos muy seguros de poder comérnoslo**.

<table>
    <tr>
        <td><img src="images_2/rewards_3.jpg" width="400" data-align="center"></td>
    </tr>
</table>

Para descontar las recompensas, procedemos así:

1. Definimos una tasa de descuento $\gamma$. Debe estar entre 0 y 1. Normalmente suele tomar valores cercanos a 0.99 o 0.95.
    * Cuanto mayor sea $\gamma$, **menor será el descuento**. Esto implica que el agente se centra más la **recompensa futura**.
    * cuanto menor sea $\gamma$, **mayor será el duescuento**. Esto implica que el agente se centra más en la **recompensa cercana**.
2. Luego, cada recompensa será descontada por $\gamma$ elevada al instante de tiempo actual. A medida que aumenta el paso de tiempo, el gato se acerca a nosotros, por lo que la recompensa futura es cada vez menos probable.

La recompensa acumulada (**descontada**) para un instante de tiempo $t$ se puede escribir como:

<table>
    <tr>
        <td><img src="images_2/rewards_2.png" width="400" data-align="center"></td>
    </tr>
</table>

$$
R(\tau) = \sum_{k=0}^{\infty} \gamma^{k} r_{t+k+1}
$$

### 2.2.6 - Enfoque de aprendizaje

**La política $\pi$ es la que define el comportamiento del agente de tal forma que sus acciones maximicen la recompensa acumulada**. Podemos verla como el **"cerebro" del agente**.

Nuestro objetivo es encontrar la política óptima $\pi^{*}$, la cual **maximiza la el retorno esperado** (i.e., la recompensa acumulada) cuando el agente actúa de acuerdo con ella. **La política óptima $\pi^{*}$ se encuentra mediante el entrenamiento.**

Hay dos posibles acercamientos para entrenar a nuestro agente a encontrar la política óptima $\pi^{*}$:
* **Directamente**, enseñando al agente que acción debe tomar en cada estado: **Policy-based methods**.
* **Indirectamente**, enseñando al agente que estado es más valioso, de tal forma que tome acciones que le lleven a estados valiosos: **Value-based methods**.

<table>
    <tr>
        <td><img src="images_2/two_approaches.png" width="600" data-align="center"></td>
    </tr>
</table>

### 2.2.1 - Cadena de Markov

### 2.2.2 - Proceso de decisión de Markov

### 2.2.3 - Funciones de valor

### 2.2.4 - Función de Bellman

### 2.2.5 - Algoritmos tradicionales

### 2.2.6 - Algoritmos basados en tablas

## 2.3 - Deep Reinforcement Learning

## 2.4 - Construyendo un agente mediante una red DQNN

### 2.4.1 - Importación de librerías

### 2.4.2 - Construcción del agente

### 2.4.3 - Construcción del método `main()`